In [23]:
#Trend analysis for each ecoregion of Nepal based on different SUs
#Input: 1. Trend Raster (all fitted and significant only)
#        2. DEM Raster (elevation, slope (degrees), aspect (degrees))

# Workflow steps : 0. SUs stratified by ecoregions > elevation > aspect > slope
#                 1. no of trend fitted pixels for each SU (>10 for valid)
#                 2. percentage of significant pixels for each SU (> %5 for valid)
#                 3. no of pixels with postive trend and negative trend
#                 4. Trend Assymetry Ratio: #n/#p (> 2 or <0.5 for valid)
#                 5. SUs that pass all criteria 
#                 6. Ecoregion trend (areal% and slope in both direction) derived from valid SUs

#starting with a simple code only considering sos  

In [24]:
# ECO_ID     ECO_NAME
# 81003     Eastern Himalayan alpine shrub and meadows
# 40115     Himalayan subtropical broadleaf forests
# 40301     Himalayan subtropical pine forests
# 40403     Western Himalayan broadleaf forests
# -9999     Rock and Ice
# 40401     Eastern Himalayan broadleaf forests
# 40166     Upper Gangetic Plains moist deciduous forests - assign value null at last for all variables
# 81021     Western Himalayan alpine shrub and Meadows
# 40701     Terai-Duar savanna and grasslands
# 40501     Eastern Himalayan subalpine conifer forests
# 40502     Western Himalayan subalpine conifer forests
# 40120     Lower Gangetic Plains moist deciduous forests - assign value null at last for all variables

In [25]:
#load libraries and data
import numpy as np  
import pandas as pd 
import matplotlib.pyplot as plt
import xarray as xr 
import rasterio as rio  
import rioxarray as rxr 
import os
import geopandas as gpd

ecoregion_rxr = rxr.open_rasterio(r"..\Data\Ecoregion_raster\ecoregions_raster.tif")

elev_rxr = rxr.open_rasterio(r"..\Data\DEM_Rasters\elevation.tif")
aspect_rxr = rxr.open_rasterio(r"..\Data\DEM_Rasters\aspect.tif")
slope_rxr = rxr.open_rasterio(r"..\Data\DEM_Rasters\slope.tif")

roi_gdf = gpd.read_file(r"../Data/roi_nepal/nepal_actual_roi.shp")
roi = roi_gdf.to_crs("EPSG:4326")

output_dir = r"../Data/Processed/Ecoregion_trends/"



In [26]:
# preprocess dem rasters, actual values to classes
# divide elevation, slope, aspect into classes

# Elevation classes: <1000: 1, 1000-2000: 2, 2000-3000: 3, 3000-4000: 4, >4000: 5
elev_class = xr.where(elev_rxr < 1000, 1, 
                      xr.where(elev_rxr < 2000, 2,
                               xr.where(elev_rxr < 3000, 3,
                                        xr.where(elev_rxr < 4000, 4, 5))))

# Slope classes: 0-2: 1, 2-15: 2, 15-30: 3, >30: 4              Reference: FAO(2006)
slope_class = xr.where(slope_rxr < 2, 1,
                       xr.where(slope_rxr < 15, 2,
                                xr.where(slope_rxr < 30, 3, 4)))

# Aspect classes: Northern (270-360, 0-90): 1, Southern (90-270): 2
aspect_class = xr.where((aspect_rxr >= 0) & (aspect_rxr <= 90), 1,
                        xr.where((aspect_rxr >= 270) & (aspect_rxr <= 360), 1,
                                 xr.where((aspect_rxr > 90) & (aspect_rxr < 270), 2, np.nan)))

In [27]:
lsp_metrics = ['sos', 'eos', 'los', 'pos']

# Initialize list to store pixel statistics for each metric
pixel_stats_list = []

for metric in lsp_metrics:
    trend_rxr = rxr.open_rasterio(r"..\Data\Trend_Rasters\mod_"+metric+"_mk_raw.tif")
    sig_trend_rxr = rxr.open_rasterio(r"..\Data\Trend_Rasters\mod_"+metric+"_mk_significant.tif")
    final_outdir = os.path.join(output_dir, f"{metric}")
    os.makedirs(final_outdir, exist_ok=True)
    
    
    #stack rasters to create a pd dataframe
    ref = sig_trend_rxr.rio.clip(roi.geometry, roi.crs, drop=True)

    raster_dict = {
        "ecoregion": ecoregion_rxr,
        "elevation": elev_class,
        "slope": slope_class,
        "aspect": aspect_class,
        "trend": trend_rxr,
        "sig_trend": sig_trend_rxr
    }

    aligned_rasters = []

    for name, raster in raster_dict.items():
        raster = raster.rio.write_crs("EPSG: 4326")
        reproj = raster.rio.reproject_match(ref)
        reproj.name = name
        reproj = reproj.squeeze('band', drop = True)
        aligned_rasters.append(reproj)

    stacked_xr = xr.merge(aligned_rasters)
    stacked_df = stacked_xr.to_dataframe().reset_index()
    
    #filter for select ecoregions
    select_ecoregions = [81003, 40115, 40301, 40403, 40401, 81021, 40701, 40501, 40502]
    stacked_df = stacked_df[stacked_df['ecoregion'].isin(select_ecoregions)]
    stacked_df1 = stacked_df[~((stacked_df['trend'] == -999) )]
    stacked_df2 = stacked_df[~((stacked_df['trend'] == -999) | (stacked_df['sig_trend'] == -999))]

    # Calculate statistics
    total_pixels = stacked_df.shape[0]
    total_trend_fitted = stacked_df1.shape[0]
    total_significant = stacked_df2.shape[0]
    
    trend_fitted_pct = (total_trend_fitted / total_pixels) * 100
    sig_from_fitted_pct = (total_significant / total_trend_fitted) * 100
    sig_from_total_pct = (total_significant / total_pixels) * 100
    
    # Print statistics (original behavior)
    print(f"\n{'='*50}")
    print(f"Statistics for {metric.upper()}")
    print(f"{'='*50}")
    print("total_pixels:", total_pixels)
    print("total trend fitted pixels:", total_trend_fitted)
    print("total significant pixels:", total_significant)
    print(f"trend fitted / total  %: {trend_fitted_pct:.2f}")
    print(f"significant / trend fitted  %: {sig_from_fitted_pct:.2f}")
    print(f"significant / total  %: {sig_from_total_pct:.2f}")
    
    # Store statistics for CSV
    pixel_stats_list.append({
        'lsp_metric': metric,
        'total_pixels': total_pixels,
        'trend_fitted_pixels': total_trend_fitted,
        'significant_pixels': total_significant,
        'trend_fitted_pct': round(trend_fitted_pct, 2),
        'sig_from_fitted_pct': round(sig_from_fitted_pct, 2),
        'sig_from_total_pct': round(sig_from_total_pct, 2)
    })

    stacked_df['trend'] = stacked_df['trend'].fillna(-999)

    su_stats = stacked_df.groupby(['ecoregion', 'elevation', 'slope', 'aspect']).agg(
        
        # 1. Trend Pixels
        n_trend_unfitted_count=('trend', lambda x: (x == -999).sum()),
        n_trend_fitted_count=('trend', lambda x: (x != -999).sum()),
        
        # 2. Significance Pixels
        n_insig_trend_count=('sig_trend', lambda x: (x == -999).sum()),
        n_sig_trend_count=('sig_trend', lambda x: ((x != -999) & (~x.isna())).sum()),
        
        # Positive and negative trends (for significant pixels only)
        positive_sig_trend_count=('sig_trend', lambda x: ((x > 0) & (x != -999)).sum()),
        negative_sig_trend_count=('sig_trend', lambda x: ((x < 0) & (x != -999)).sum()),
        
        # Mean values (excluding -999 and NaN)
        positive_sig_trend_mean=('sig_trend', lambda x: x[(x > 0) & (x != -999)].mean() if len(x[(x > 0) & (x != -999)]) > 0 else np.nan),
        negative_sig_trend_mean=('sig_trend', lambda x: x[(x < 0) & (x != -999)].mean() if len(x[(x < 0) & (x != -999)]) > 0 else np.nan),
        all_sig_trend_mean=('sig_trend', lambda x: x[(x != -999) & (~x.isna())].mean())
    ).reset_index()

    # Add derived metrics based on your workflow
    su_stats['percent_sig_pixels'] = (su_stats['n_sig_trend_count'] / su_stats['n_trend_fitted_count']) * 100
    su_stats['trend_asymmetry_ratio'] = su_stats['positive_sig_trend_count'] / su_stats['negative_sig_trend_count']


    #based on the set criteria we divide SUs into either 'valid' or 'invalid' category in lsp_change
    su_stats['lsp_change'] = np.where(
        (su_stats['n_trend_fitted_count'] > 10) &  # At least 10 fitted pixels
        (su_stats['percent_sig_pixels'] > 5) &      # At least 5% significant
        ((su_stats['trend_asymmetry_ratio'] > 2) | (su_stats['trend_asymmetry_ratio'] < 0.5)),  # Strong asymmetry
        1,                  #means yes or valid
        0                   #means no or invalid
    )

    su_stats.to_csv(os.path.join(final_outdir, "All_SU_Stats_"+metric+".csv"), index=False)

    #from this step we drop all SUs with invalid lsp_change
    # all % are calculated based on the ecoregions area (total pixels of that ecoregions)

    ecr_count = su_stats.groupby('ecoregion')[['n_trend_fitted_count', 'n_trend_unfitted_count']].sum().reset_index()
    ecr_count['total_pixels'] = ecr_count['n_trend_fitted_count'] + ecr_count['n_trend_unfitted_count']
    ecr_count = ecr_count.drop(columns=['n_trend_fitted_count', 'n_trend_unfitted_count'])

    su_filtered = (su_stats[su_stats['lsp_change'] == 1]).drop(columns=['lsp_change'])
    ecr_stats = su_filtered.groupby('ecoregion').agg(
        n_trend_fitted_count = ('n_trend_fitted_count', 'sum'),
        n_sig_trend_count=('n_sig_trend_count', 'sum'),
        positive_sig_trend_count=('positive_sig_trend_count', 'sum'),
        negative_sig_trend_count=('negative_sig_trend_count', 'sum'),
        positive_sig_trend_mean=('positive_sig_trend_mean', 'mean'),
        negative_sig_trend_mean=('negative_sig_trend_mean', 'mean'),
        all_sig_trend_mean=('all_sig_trend_mean', 'mean')
    )

    ecr_stats = pd.merge(ecr_stats, ecr_count, on='ecoregion', how = 'inner')    

    ecr_stats['percent_trend_fit_px'] = (ecr_stats['n_trend_fitted_count'] / ecr_stats['total_pixels']) * 100
    ecr_stats['percent_significant_valid_px'] = (ecr_stats['n_sig_trend_count'] / ecr_stats['n_trend_fitted_count']) * 100
    ecr_stats['percent_positive_valid_px'] = (ecr_stats['positive_sig_trend_count'] / ecr_stats['n_trend_fitted_count']) * 100
    ecr_stats['percent_negative_valid_px'] = (ecr_stats['negative_sig_trend_count'] / ecr_stats['n_trend_fitted_count']) * 100
    ecr_stats['trend_asymmetry_ratio'] = ecr_stats['positive_sig_trend_count'] / ecr_stats['negative_sig_trend_count']

    #calculate net area % and net trend mean for ecoregions with notably asymmetric trend
    ecr_stats['net_area_percent'] = np.where(ecr_stats['trend_asymmetry_ratio'] > 2,
        ecr_stats['percent_positive_valid_px'] - ecr_stats['percent_negative_valid_px'],
        np.where(ecr_stats['trend_asymmetry_ratio'] < 0.5,
            ecr_stats['percent_positive_valid_px'] - ecr_stats['percent_negative_valid_px'],
            0
        )
    )
    
    ecr_stats = (ecr_stats[['ecoregion',
        'percent_positive_valid_px',
        'percent_negative_valid_px',
        'positive_sig_trend_mean',
        'negative_sig_trend_mean',
        'trend_asymmetry_ratio',
        'net_area_percent']])
    ecr_stats = ecr_stats.round(2)
    ecr_stats.to_csv(os.path.join(final_outdir, "ecoregion_stats.csv"), index=False)

# Convert to DataFrame and save as CSV
pixel_stats_df = pd.DataFrame(pixel_stats_list)
pixel_stats_df.to_csv(os.path.join(output_dir, "pixel_statistics_summary.csv"), index=False)


C:\Users\A S U S\AppData\Local\Temp\ipykernel_15692\3232685818.py:34: FutureWarning: In a future version of xarray the default value for compat will change from compat='no_conflicts' to compat='override'. This is likely to lead to different results when combining overlapping variables with the same name. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set compat explicitly.
  stacked_xr = xr.merge(aligned_rasters)



Statistics for SOS
total_pixels: 2565421
total trend fitted pixels: 2044237
total significant pixels: 159563
trend fitted / total  %: 79.68
significant / trend fitted  %: 7.81
significant / total  %: 6.22


C:\Users\A S U S\AppData\Local\Temp\ipykernel_15692\3232685818.py:34: FutureWarning: In a future version of xarray the default value for compat will change from compat='no_conflicts' to compat='override'. This is likely to lead to different results when combining overlapping variables with the same name. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set compat explicitly.
  stacked_xr = xr.merge(aligned_rasters)



Statistics for EOS
total_pixels: 2565421
total trend fitted pixels: 2045288
total significant pixels: 154988
trend fitted / total  %: 79.73
significant / trend fitted  %: 7.58
significant / total  %: 6.04


C:\Users\A S U S\AppData\Local\Temp\ipykernel_15692\3232685818.py:34: FutureWarning: In a future version of xarray the default value for compat will change from compat='no_conflicts' to compat='override'. This is likely to lead to different results when combining overlapping variables with the same name. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set compat explicitly.
  stacked_xr = xr.merge(aligned_rasters)



Statistics for LOS
total_pixels: 2565421
total trend fitted pixels: 2045288
total significant pixels: 191144
trend fitted / total  %: 79.73
significant / trend fitted  %: 9.35
significant / total  %: 7.45


C:\Users\A S U S\AppData\Local\Temp\ipykernel_15692\3232685818.py:34: FutureWarning: In a future version of xarray the default value for compat will change from compat='no_conflicts' to compat='override'. This is likely to lead to different results when combining overlapping variables with the same name. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set compat explicitly.
  stacked_xr = xr.merge(aligned_rasters)



Statistics for POS
total_pixels: 2565421
total trend fitted pixels: 2045288
total significant pixels: 513054
trend fitted / total  %: 79.73
significant / trend fitted  %: 25.08
significant / total  %: 20.00


In [28]:
# Topographic Driver Analysis - Which factors control LSP change direction?
# For each ecoregion and metric, test if Elevation, Slope, or Aspect significantly influence trend asymmetry

from scipy.stats import spearmanr, mannwhitneyu

# Initialize list to store driver results for all metrics
all_driver_results = []

for metric in lsp_metrics:
    final_outdir = os.path.join(output_dir, f"{metric}")
    
    # Load the SU stats for this metric
    su_stats = pd.read_csv(os.path.join(final_outdir, "All_SU_Stats_"+metric+".csv"))
    
    # Step 1: Create normalized asymmetry index (avoids division by zero)
    # Range: -1 (all positive trends) to +1 (all negative trends)
    su_stats['asymmetry_index'] = (
        (su_stats['positive_sig_trend_count'] - su_stats['negative_sig_trend_count']) /
        (su_stats['positive_sig_trend_count'] + su_stats['negative_sig_trend_count'])
    )
    
    # Step 2: Filter for valid SUs only
    valid_sus = su_stats[su_stats['lsp_change'] == 1].dropna(subset=['asymmetry_index'])
    
    # Step 3: Test each ecoregion
    driver_results = []
    
    for eco_id, group in valid_sus.groupby('ecoregion'):
        if len(group) < 5:  # Skip if too few SUs
            continue
        
        # --- ELEVATION: Spearman correlation ---
        rho_elev, p_elev = spearmanr(group['elevation'], group['asymmetry_index'])
        is_elev_driver = (p_elev < 0.05) and (abs(rho_elev) > 0.1)
        
        # --- SLOPE: Spearman correlation ---
        rho_slope, p_slope = spearmanr(group['slope'], group['asymmetry_index'])
        is_slope_driver = (p_slope < 0.05) and (abs(rho_slope) > 0.1)
        
        # --- ASPECT: Mann-Whitney with directional effect size ---
        north = group[group['aspect'] == 1]['asymmetry_index']
        south = group[group['aspect'] == 2]['asymmetry_index']
        
        if len(north) > 5 and len(south) > 5:
            u_stat, p_aspect = mannwhitneyu(north, south)
            
            # Calculate effect size r = |Z| / sqrt(N)
            n1, n2 = len(north), len(south)
            mu = n1 * n2 / 2
            sigma = np.sqrt(n1 * n2 * (n1 + n2 + 1) / 12)
            z_score = (u_stat - mu) / sigma
            r_magnitude = abs(z_score) / np.sqrt(n1 + n2)
            
            # Assign direction: + if North has higher asymmetry, - if South higher
            r_aspect = r_magnitude if north.median() > south.median() else -r_magnitude
            
            is_aspect_driver = (p_aspect < 0.05) and (abs(r_aspect) > 0.1)
        else:
            r_aspect, p_aspect, is_aspect_driver = 0, 1.0, False
        
        # Store results
        driver_results.append({
            'lsp_metric': metric,
            'ecoregion': eco_id,
            'elev_r': round(rho_elev, 3),
            'slope_r': round(rho_slope, 3),
            'aspect_r': round(r_aspect, 3),
            'elev_p': round(p_elev, 4),
            'slope_p': round(p_slope, 4),
            'aspect_p': round(p_aspect, 4),
            'is_elev_driver': 'YES' if is_elev_driver else 'NO',
            'is_slope_driver': 'YES' if is_slope_driver else 'NO',
            'is_aspect_driver': 'YES' if is_aspect_driver else 'NO'
        })
    
    # Add to overall results
    all_driver_results.extend(driver_results)
    
    # Step 4: Create results table for this metric
    drivers_df = pd.DataFrame(driver_results)
    print(f"\n{'='*50}")
    print(f"Topographic Drivers for {metric.upper()}")
    print(f"{'='*50}")
    print("Criteria: p < 0.05 AND |r| > 0.1")
    print("\nInterpretation:")
    print("  + value = Higher class/North aspect → more positive trends")
    print("  - value = Higher class/South aspect → more negative trends\n")
    display(drivers_df)
    
    # Save results for this metric
    drivers_df.to_csv(os.path.join(final_outdir, "Topographic_Drivers.csv"), index=False)

# Save combined results for all metrics
all_drivers_df = pd.DataFrame(all_driver_results)
all_drivers_df.to_csv(os.path.join(output_dir, "Topographic_Drivers_All_Metrics.csv"), index=False)


Topographic Drivers for SOS
Criteria: p < 0.05 AND |r| > 0.1

Interpretation:
  + value = Higher class/North aspect → more positive trends
  - value = Higher class/South aspect → more negative trends



,lsp_metric,ecoregion,elev_r,slope_r,aspect_r,elev_p,slope_p,aspect_p,is_elev_driver,is_slope_driver,is_aspect_driver
0,sos,40115,-0.722,-0.771,0.000,0.0671,0.0424,1.0000,NO,YES,NO
1,sos,40301,0.193,-0.061,0.000,0.4903,0.8294,1.0000,NO,NO,NO
2,sos,40401,0.669,0.020,-0.037,0.0002,0.9224,0.8711,YES,NO,NO
3,sos,40403,0.339,0.122,0.000,0.3717,0.7549,1.0000,NO,NO,NO
4,sos,40501,0.865,0.305,0.142,0.0000,0.2344,0.5868,YES,NO,NO
5,sos,40502,0.208,-0.535,0.000,0.5915,0.1381,1.0000,NO,NO,NO
6,sos,81003,0.372,-0.441,0.000,0.4679,0.3809,1.0000,NO,NO,NO



Topographic Drivers for EOS
Criteria: p < 0.05 AND |r| > 0.1

Interpretation:
  + value = Higher class/North aspect → more positive trends
  - value = Higher class/South aspect → more negative trends



,lsp_metric,ecoregion,elev_r,slope_r,aspect_r,elev_p,slope_p,aspect_p,is_elev_driver,is_slope_driver,is_aspect_driver
0,eos,40301,0.329,0.569,0.000,0.2721,0.0426,1.0000,NO,YES,NO
1,eos,40401,0.157,-0.100,-0.480,0.5085,0.6739,0.0347,NO,NO,YES
2,eos,40403,0.359,-0.707,0.000,0.5528,0.1817,1.0000,NO,NO,NO
3,eos,40501,-0.334,0.453,0.000,0.2885,0.1397,1.0000,NO,NO,NO
4,eos,40502,0.881,-0.431,-0.345,0.0000,0.1241,0.2195,YES,NO,NO
5,eos,81003,-0.822,0.243,0.000,0.0066,0.5294,1.0000,YES,NO,NO
6,eos,81021,-0.064,-0.569,0.499,0.8150,0.0214,0.0519,NO,YES,NO



Topographic Drivers for LOS
Criteria: p < 0.05 AND |r| > 0.1

Interpretation:
  + value = Higher class/North aspect → more positive trends
  - value = Higher class/South aspect → more negative trends



,lsp_metric,ecoregion,elev_r,slope_r,aspect_r,elev_p,slope_p,aspect_p,is_elev_driver,is_slope_driver,is_aspect_driver
0,los,40115,-0.365,-0.477,0.000,0.3339,0.1946,1.0000,NO,NO,NO
1,los,40301,-0.356,-0.269,-0.322,0.1341,0.2656,0.1774,NO,NO,NO
2,los,40401,-0.698,0.095,-0.225,0.0009,0.6997,0.3469,YES,NO,NO
3,los,40403,-0.116,-0.591,0.000,0.7489,0.0719,1.0000,NO,NO,NO
4,los,40501,-0.827,0.215,0.257,0.0000,0.4065,0.3117,YES,NO,NO
5,los,81003,-0.064,0.360,0.000,0.8518,0.2774,1.0000,NO,NO,NO
6,los,81021,-0.775,0.170,0.000,0.0239,0.6871,1.0000,YES,NO,NO



Topographic Drivers for POS
Criteria: p < 0.05 AND |r| > 0.1

Interpretation:
  + value = Higher class/North aspect → more positive trends
  - value = Higher class/South aspect → more negative trends



C:\Users\A S U S\AppData\Local\Temp\ipykernel_15692\1844569235.py:33: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  rho_elev, p_elev = spearmanr(group['elevation'], group['asymmetry_index'])


,lsp_metric,ecoregion,elev_r,slope_r,aspect_r,elev_p,slope_p,aspect_p,is_elev_driver,is_slope_driver,is_aspect_driver
0,pos,40115,0.000,-0.752,-0.039,1.0000,0.0008,0.9163,NO,YES,NO
1,pos,40301,-0.330,-0.038,0.020,0.1002,0.8542,0.9385,NO,NO,NO
2,pos,40401,0.075,0.324,-0.189,0.6734,0.0615,0.2758,NO,NO,NO
3,pos,40403,0.470,0.186,0.136,0.0154,0.3633,0.5050,YES,NO,NO
4,pos,40501,0.099,0.081,-0.315,0.6394,0.7013,0.1204,NO,NO,NO
5,pos,40502,-0.918,0.324,-0.203,0.0000,0.1628,0.3847,YES,NO,NO
6,pos,40701,NaN,-0.837,0.000,NaN,0.0378,1.0000,NO,YES,NO
7,pos,81003,0.692,-0.394,0.114,0.0015,0.1057,0.6582,YES,NO,NO
8,pos,81021,0.093,0.074,-0.082,0.7235,0.7772,0.7727,NO,NO,NO


In [29]:
# Ecoregion Summary - Trend Fit Status and Dominant Directions (Metric-wise)
# Summarizes key statistics for each ecoregion and LSP metric combination

# Initialize list to store ecoregion-metric combinations
ecoregion_metric_summary = []

for metric in lsp_metrics:
    final_outdir = os.path.join(output_dir, f"{metric}")
    
    # Load the SU stats for this metric
    su_stats = pd.read_csv(os.path.join(final_outdir, "All_SU_Stats_"+metric+".csv"))
    
    # Calculate ecoregion-level statistics for this metric
    for eco_id in su_stats['ecoregion'].unique():
        eco_data = su_stats[su_stats['ecoregion'] == eco_id]
        
        # 1. Number of SUs with at least 10 trend fitted pixels
        n_sus = len(eco_data[eco_data['n_trend_fitted_count'] > 10])
        
        # 2. Percentage of pixels with fitted trend (total for ecoregion)
        total_pixels = eco_data['n_trend_fitted_count'].sum() + eco_data['n_trend_unfitted_count'].sum()
        fitted_pixels = eco_data['n_trend_fitted_count'].sum()
        percent_fitted = (fitted_pixels / total_pixels * 100) if total_pixels > 0 else 0
        
        # 3. Percentage of pixels with significant trend from valid SUs
        valid_sus = eco_data[eco_data['lsp_change'] == 1]
        total_fitted = eco_data['n_trend_fitted_count'].sum()
        sig_valid_pixels = valid_sus['n_sig_trend_count'].sum()
        percent_sig_valid = (sig_valid_pixels / total_fitted * 100) if total_fitted > 0 else 0
        
        # 4. Dominant Trend Direction for this metric
        if len(valid_sus) > 0:
            pos_count = valid_sus['positive_sig_trend_count'].sum()
            neg_count = valid_sus['negative_sig_trend_count'].sum()
            asymmetry_ratio = pos_count / neg_count if neg_count > 0 else np.inf
            
            # Determine dominant direction based on asymmetry
            if asymmetry_ratio > 2:  # Predominantly positive trends
                if metric in ['sos', 'eos']:
                    direction = 'Delay'
                else:  # los, pos
                    direction = 'Increasing'
            elif asymmetry_ratio < 0.5:  # Predominantly negative trends
                if metric in ['sos', 'eos']:
                    direction = 'Advance'
                else:  # los, pos
                    direction = 'Decreasing'
            else:  # No clear asymmetry
                direction = 'Mixed'
        else:
            direction = 'No Valid SUs'
        
        # Store all information for this ecoregion-metric combination
        ecoregion_metric_summary.append({
            'lsp_metric': metric.upper(),
            'ecoregion': eco_id,
            'n_sus_with_min_10_fitted': n_sus,
            'percent_fitted_pixels': round(percent_fitted, 2),
            'percent_sig_valid_pixels': round(percent_sig_valid, 2),
            'dominant_direction': direction
        })

# Convert to DataFrame
summary_df = pd.DataFrame(ecoregion_metric_summary)

# Sort by metric and then by ecoregion
summary_df = summary_df.sort_values(['lsp_metric', 'ecoregion']).reset_index(drop=True)

#display(summary_df)

# Save to CSV
#summary_df.to_csv(r"..\Data\Processed\Trend_fit\SummaryTrendFit.csv", index=False)
#print(f"\nSummary saved to: Data\Processed\Trend_fit\SummaryTrendFit.csv")